# Hummingbot Status Report via Telegram

## 🤖 Overview
This notebook uses the official `hummingbot-api-client` to retrieve portfolio state and active bot status, then sends a formatted report via Telegram. Designed for automated task execution.

## 🎯 Features
- **Official API Client**: Uses `hummingbot-api-client` package
- **Portfolio State**: Current balances across all exchanges
- **Active Bots**: Running strategies with performance metrics
- **Telegram Delivery**: Professional HTML-formatted reports

## 📋 Prerequisites
- Hummingbot running with API enabled (localhost:8000)
- Telegram bot configured in .env file
- `hummingbot-api-client` package installed

In [ ]:
# 🔌 Setup and Imports
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

from datetime import datetime, timezone
from hummingbot_api_client import HummingbotAPIClient
from core.notifiers import NotificationManager, NotificationMessage

load_dotenv()

print("✅ Hummingbot Status Reporter initialized")
print("🤖 Using official hummingbot-api-client")

In [ ]:
# 📊 Fetch Portfolio State and Active Bots
portfolio_state = None
active_bots_data = None
portfolio_value = 0.0
top_balances = []
active_bots = []
api_connected = False

print("📊 Fetching data from Hummingbot API...")

async with HummingbotAPIClient(base_url="http://localhost:8000") as client:
    try:
        # Get portfolio state
        try:
            portfolio_state = await client.portfolio.get_state()
            if portfolio_state and 'master_account' in portfolio_state:
                api_connected = True
                print("✅ Portfolio state retrieved")
                
                # Process portfolio data
                master_account = portfolio_state['master_account']
                
                for exchange, assets in master_account.items():
                    if isinstance(assets, list):
                        for asset_info in assets:
                            token = asset_info.get('token', 'Unknown')
                            units = float(asset_info.get('units', 0))
                            price = float(asset_info.get('price', 0))
                            value = float(asset_info.get('value', 0))
                            
                            # Only include significant balances (> $1)
                            if units > 0 and value >= 1.0:
                                top_balances.append({
                                    'asset': token,
                                    'exchange': exchange,
                                    'units': units,
                                    'price': price,
                                    'value': value
                                })
                                portfolio_value += value
                
                # Sort by value descending
                top_balances.sort(key=lambda x: x['value'], reverse=True)
                print(f"✅ Portfolio: ${portfolio_value:,.2f} in {len(top_balances)} positions")
        
        except Exception as e:
            print(f"⚠️  Portfolio state: {e}")
        
        # Get active bots status
        try:
            active_bots_data = await client.bot_orchestration.get_active_bots_status()
            if active_bots_data and active_bots_data.get('status') == 'success':
                print("✅ Active bots data retrieved")
                
                bots_data = active_bots_data.get('data', {})
                
                for bot_name, bot_info in bots_data.items():
                    if isinstance(bot_info, dict) and 'status' in bot_info:
                        bot_status = bot_info.get('status', 'Unknown')
                        performance = bot_info.get('performance', {})
                        
                        # Extract performance metrics
                        total_pnl = 0.0
                        total_volume = 0.0
                        strategies_count = len(performance)
                        
                        for strategy_name, strategy_perf in performance.items():
                            if isinstance(strategy_perf, dict) and 'performance' in strategy_perf:
                                perf_data = strategy_perf['performance']
                                total_pnl += float(perf_data.get('global_pnl_quote', 0))
                                total_volume += float(perf_data.get('volume_traded', 0))
                        
                        active_bots.append({
                            'name': bot_name,
                            'status': bot_status,
                            'strategies_count': strategies_count,
                            'total_pnl': total_pnl,
                            'total_volume': total_volume
                        })
                
                print(f"✅ Active bots: {len(active_bots)} running")
        
        except Exception as e:
            print(f"⚠️  Active bots: {e}")
            
    except Exception as e:
        print(f"❌ API Connection failed: {e}")

In [ ]:
# 📱 Generate and Send Telegram Report
notification_manager = NotificationManager()

if 'telegram' not in notification_manager.get_enabled_notifiers():
    print("❌ Telegram not configured in .env file")
else:
    print("📱 Generating Telegram report...")
    
    # Generate timestamp
    report_time = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')
    
    # Build Telegram report with HTML formatting
    telegram_report = f"""<b>🤖 Hummingbot Status Report</b>
🕒 <i>{report_time}</i>

<b>📊 System Status</b>
{'✅' if api_connected else '❌'} API: {'Connected' if api_connected else 'Failed'}
🤖 Bots: {len(active_bots)} active

<b>💰 Portfolio Summary</b>
💵 <b>Total Value: ${portfolio_value:,.2f}</b>
📈 Positions: {len(top_balances)}"""
    
    # Add top 10 holdings (simplified - only USD value)
    if top_balances:
        telegram_report += f"\n\n<b>🏆 Top Holdings:</b>\n"
        for i, balance in enumerate(top_balances[:10], 1):
            asset = balance['asset']
            exchange = balance['exchange']
            value = balance['value']
            
            telegram_report += f"<code>{i:2d}. {asset:>8s} | ${value:>10,.2f} | {exchange}</code>\n"
    
    # Add active bots section (simplified format)
    telegram_report += f"\n<b>⚙️ Active Trading Bots</b>\n"
    if active_bots:
        for bot in active_bots:
            name = bot['name']
            status = bot['status']
            strategies_count = bot['strategies_count']
            pnl = bot['total_pnl']
            volume = bot['total_volume']
            
            status_emoji = "🟢" if status.lower() == 'running' else "🟡"
            pnl_emoji = "📈" if pnl > 0 else "📉" if pnl < 0 else "➖"
            
            # Single line format: Bot | Status | P&L | Volume
            telegram_report += f"{status_emoji} <b>{name}</b> | {status} | {pnl_emoji} ${pnl:+.2f}"
            if volume > 0:
                telegram_report += f" | 📊 ${volume:,.0f}"
            telegram_report += f"\n"
            
            # Show strategy details below if multiple strategies
            if strategies_count > 1:
                telegram_report += f"   <i>└── {strategies_count} strategies running</i>\n"
            telegram_report += f"\n"
    else:
        telegram_report += f"⚠️ No active bots detected\n"
    
    # Add footer
    telegram_report += f"\n<i>📊 QuantsLab • localhost:8000</i>"
    
    # Send report
    try:
        telegram_notifier = notification_manager.get_notifier('telegram')
        
        notification = NotificationMessage(
            title="🤖 Hummingbot Status Report",
            message=telegram_report,
            level="info"
        )
        
        success = await telegram_notifier.send_notification(notification)
        
        if success:
            print("✅ Report sent to Telegram successfully!")
            
            # Calculate total P&L
            total_pnl = sum(bot['total_pnl'] for bot in active_bots)
            print(f"📊 Summary: {len(active_bots)} bots • ${portfolio_value:,.2f} • P&L: ${total_pnl:+.2f}")
        else:
            print("❌ Failed to send Telegram report")
            
    except Exception as e:
        print(f"❌ Telegram error: {e}")

In [ ]:
# 🎉 Summary
print(f"\n🎉 Hummingbot monitoring complete!")
print(f"🤖 Active Bots: {len(active_bots)}")
print(f"💰 Portfolio: ${portfolio_value:,.2f}")
if active_bots:
    total_pnl = sum(bot['total_pnl'] for bot in active_bots)
    total_volume = sum(bot['total_volume'] for bot in active_bots)
    print(f"📈 Total P&L: ${total_pnl:+.2f}")
    print(f"📊 Total Volume: ${total_volume:,.0f}")
print(f"📱 Telegram: {'✅ Sent' if 'telegram' in notification_manager.get_enabled_notifiers() else '❌ Not configured'}")